In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
#%ls
#%cd 'drive/MyDrive/Colab Notebooks/DivNet'

In [2]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import helperfunc as hf
import torch
import trainers as t
import Networks as n
import pickle
import torch.nn as nn
#!pip install ipython-autotime
%load_ext autotime
print(torch.__version__)

1.7.1
time: 381 µs (started: 2021-03-03 12:03:21 -05:00)


In [3]:
##load data
In = np.load('data/nn_output_10.npy')
out = np.load('data/nn_input_10.npy')
N,Dx = np.shape(In)
N,Dy =np.shape(out)
print('Input shape: ', np.shape(In))
print('Output shape: ',np.shape(out))

## Normalizing
for i in range(Dx):
    if np.min(In[:,i])<-200:
        new_data = (In[:,i]-np.mean(In[:,i]))/(np.std(In[:,i]))
        In[:,i]=new_data
        
from sklearn.model_selection import train_test_split
In, X_test, out, y_test = train_test_split(In, out, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

Input shape:  (1000000, 24)
Output shape:  (1000000, 27)
time: 3.31 s (started: 2021-03-03 12:03:25 -05:00)


In [4]:
config_single ={
         "lr":1e-3,
         "epochs": 100,
         "batch_size": 32,
         "iters": 1000,#int(N/32),
         "input_dim" : Dx,
         "output_dim" : Dy,
         "hidden1" :700,
         "hidden2" :700,
         "display_interval":20,
         "print_it" : True,
         "save":True
    }

time: 461 µs (started: 2021-03-03 12:03:28 -05:00)


In [5]:
config_section ={
        "init_select":4,
        "runs":50,
         "lr":2e-3,
         "epochs": 50,
         "batch_size": 32,
         "iters": 100,#int(N/32),
         "input_dim" : Dx,
         "output_dim" : Dy,
         "hidden1" :30, #
         "hidden2" :30,
         "display_interval":30,
         "print_it" : True,
         "c":50,
         "ref_s": 7,#int(np.ceil(Dx/Dy)),
         "ref_v":np.var(In),
         "phi_1":1.0,
         "phi_2":1e-1,
         "phi_3":1e-3,
         "beta":0.1,
         "save":False,
         "dim":6#np.argmin(cor) #fixed value to avaoid calculating correlation every time
    }

time: 125 ms (started: 2021-03-03 12:03:28 -05:00)


In [6]:
config_tiny ={
        "input_dim":1,
        "output_dim":1,
        "hidden1": 3,
        "hidden2":2,
        "lr":2e-3,
        "epochs": 100,
        "batch_size": 400,
        "iters": int(N/400),
        "display_interval":10,
        "print_it":True,
        "dim":config_section["dim"]
}

time: 564 µs (started: 2021-03-03 12:03:29 -05:00)


In [7]:
device =torch.device('cpu')
PATH="30x30model/tiny_dict_model.pt"

NNt = n.tiny_NN(config_tiny).to(device)
NNt.load_state_dict(torch.load(PATH, map_location=device))

with open('30x30model/params30.pkl','rb') as f:  
    SN, bn, on = pickle.load(f)
NN={}
for i in range(SN):
    PATH="30x30model/section_dict_model30"+str(i)+".pt"
    NN[str(i)]=n.section_NN(config_section)
    NN[str(i)].load_state_dict(torch.load(PATH))

PATH="30x30model/single_dict_model.pt"

NNl = n.Large_NN(config_single)
NNl.load_state_dict(torch.load(PATH))

<All keys matched successfully>

time: 34.6 ms (started: 2021-03-03 12:03:31 -05:00)


In [8]:
def test(NNt,NN,X_test, y_test,SN,dim,on,bn):
    criterion = nn.MSELoss()
    #NNt.eval()
    N,D = np.shape(X_test)
    X_test = torch.tensor(X_test, dtype=torch.float,requires_grad=False)#.to(device)
    y_test = torch.tensor(y_test, dtype=torch.float)#.to(device) #  tensor
    selectors = NNt(torch.reshape(X_test[:,dim],(N,1))).cpu().detach().numpy()
    loss = 0 
    q = [int(np.round(x)) for x in selectors]
    print('calculating loss')

    #for i in range(SN):
    #  NN[str(i)].eval()
    for j in range(N):
        ypred=NN[str(q[j])](X_test[j])
        loss += criterion(ypred, y_test[j]).item()  

    return loss/N

time: 1.23 ms (started: 2021-03-03 12:03:36 -05:00)


In [9]:
def testl(NNl,X_test, y_test):
    criterion = nn.MSELoss()
    N,D = np.shape(X_test)
    #NNl.to(device)
    X_test = torch.tensor(X_test, dtype=torch.float,requires_grad=False)#.to(device)
    y_test = torch.tensor(y_test, dtype=torch.float)#.to(device) #  tensor
    loss = 0 
    print('calculating loss')
    NNl.eval()
    ypred=NNl(X_test)
    loss += criterion(ypred, y_test).item()

    return loss

time: 779 µs (started: 2021-03-03 12:03:36 -05:00)
